In [16]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk
from sklearn.model_selection import cross_val_score

In [2]:
nlp = spacy.load('en')

In [3]:
asv = pd.read_csv("t_asv.csv")

In [4]:
asv["Author"].unique()

array(['Matthew', 'Mark', 'Luke', 'John', 'Paul', 'Unknown', 'James',
       'Peter', 'Jude'], dtype=object)

In [5]:
asv_Matthew = asv[asv["Author"] == "Matthew"]
asv_Mark = asv[asv["Author"] == "Mark"]
asv_Luke = asv[asv["Author"] == "Luke"]
asv_John = asv[asv["Author"] == "John"]
asv_Paul = asv[asv["Author"] == "Paul"]
asv_Unknown = asv[asv["Author"] == "Unknown"]
asv_James = asv[asv["Author"] == "James"]
asv_Peter = asv[asv["Author"] == "Peter"]
asv_Jude = asv[asv["Author"] == "Jude"]
    

In [6]:
asv_Matthew_doc = nlp(''.join(asv_Matthew["Text"]))
asv_Matthew_sents = [[sent, "Matthew"] for sent in asv_Matthew_doc.sents]

asv_Mark_doc = nlp(''.join(asv_Mark["Text"]))
asv_Mark_sents = [[sent, "Mark"] for sent in asv_Mark_doc.sents]

asv_Luke_doc = nlp(''.join(asv_Luke["Text"]))
asv_Luke_sents = [[sent, "Luke"] for sent in asv_Luke_doc.sents]

asv_John_doc = nlp(''.join(asv_John["Text"]))
asv_John_sents = [[sent, "John"] for sent in asv_John_doc.sents]

asv_Paul_doc = nlp(''.join(asv_Paul["Text"]))
asv_Paul_sents = [[sent, "Paul"] for sent in asv_Paul_doc.sents]

asv_Unknown_doc = nlp(''.join(asv_Unknown["Text"]))
asv_Unknown_sents = [[sent, "Unknown"] for sent in asv_Unknown_doc.sents]

asv_James_doc = nlp(''.join(asv_James["Text"]))
asv_James_sents = [[sent, "James"] for sent in asv_James_doc.sents]

asv_Peter_doc = nlp(''.join(asv_Peter["Text"]))
asv_Peter_sents = [[sent, "Peter"] for sent in asv_Peter_doc.sents]

asv_Jude_doc = nlp(''.join(asv_Jude["Text"]))
asv_Jude_sents = [[sent, "Jude"] for sent in asv_Jude_doc.sents]

[[Jude, a servant of Jesus Christ, and brother of James, to them that are called, beloved in God the Father, and kept for Jesus Christ:Mercy unto you and peace and love be multiplied.,
  'Jude'],
 [Beloved, while I was giving all diligence to write unto you of our common salvation, I was constrained to write unto you exhorting you to contend earnestly for the faith which was once for all delivered unto the saints.,
  'Jude'],
 [For there are certain men crept in privily, `even' they who were of old written of beforehand unto this condemnation, ungodly men, turning the grace of our God into lasciviousness, and denying our only Master and Lord, Jesus Christ.,
  'Jude'],
 [Now I desire to put you in remembrance, though ye know all things once for all, that the Lord, having saved a people out of the land of Egypt, afterward destroyed them that believed not.,
  'Jude'],
 [And angels that kept not their own principality, but left their proper habitation, he hath kept in everlasting bonds und

In [22]:
df = pd.DataFrame(asv_Matthew_doc + asv_Mark_doc + asv_Luke_doc + asv_John_doc + asv_Paul_doc + asv_Unknown_doc
                  + asv_James_doc + asv_Peter_doc + asv_Jude_doc)

TypeError: unsupported operand type(s) for +: 'spacy.tokens.doc.Doc' and 'spacy.tokens.doc.Doc'

In [7]:
df2 = pd.DataFrame(asv_Matthew_sents + asv_Mark_sents + asv_Luke_sents + asv_John_sents + asv_Paul_sents + asv_Unknown_sents
                  + asv_James_sents + asv_Peter_sents + asv_Jude_sents)

In [8]:
df2.head()

,0,1
0,"(The, book, of, the, generation, of, Jesus, Ch...",Matthew
1,"(Abraham, begat, Isaac, ;, and, Isaac, begat, ...",Matthew
2,"(Jesse, begat)",Matthew
3,"(David, the, king, .)",Matthew
4,"(And, David, begat, Solomon, of, her, `, that,...",Matthew


# TF-IDF

In [9]:
from sklearn.model_selection import train_test_split

X = asv["Text"]
y = asv["Author"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Naive Bayes:
text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),
])

# Linear SVC:
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

#Logistic Regression
text_clf_lr = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LogisticRegression()),
])


## Naive Bayes

In [11]:
#run Naive Bayes first
text_clf_nb.fit(X_train, y_train)

#run predictions
prednb = text_clf_nb.predict(X_test)


In [12]:
df3 = pd.DataFrame(metrics.confusion_matrix(y_test,prednb), 
      index=['James','John','Jude',
            'Luke','Mark','Matthew','Paul',
            'Peter','Unknown'], 
     columns=['James','John','Jude',
            'Luke','Mark','Matthew','Paul',
            'Peter','Unknown'])
df3

,James,John,Jude,Luke,Mark,Matthew,Paul,Peter,Unknown
James,0,0,0,5,0,0,17,0,0
John,0,156,0,130,0,1,59,0,0
Jude,0,0,0,1,0,0,4,0,0
Luke,0,14,0,450,0,3,73,0,0
Mark,0,7,0,153,0,0,14,0,0
Matthew,0,23,0,203,0,7,29,0,0
Paul,0,6,0,55,0,0,465,0,0
Peter,0,0,0,9,0,0,32,0,0
Unknown,0,6,0,30,0,0,38,0,0


In [13]:
#sns.heatmap(df3, cmap='Blues',annot=True, )

In [47]:
print(metrics.classification_report(y_test,prednb))

c:\users\sakok\miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

       James       0.00      0.00      0.00        41
        John       0.80      0.40      0.53       468
        Jude       0.00      0.00      0.00         7
        Luke       0.43      0.88      0.58       715
        Mark       0.00      0.00      0.00       221
     Matthew       0.86      0.03      0.06       359
        Paul       0.64      0.88      0.74       672
       Peter       0.00      0.00      0.00        42
     Unknown       0.00      0.00      0.00       102

   micro avg       0.54      0.54      0.54      2627
   macro avg       0.30      0.24      0.21      2627
weighted avg       0.54      0.54      0.45      2627



In [48]:
print(metrics.accuracy_score(y_test,prednb))

0.5393985534830605


In [18]:
cross_val_score(text_clf_nb, X_test, y_test, cv=10)

c:\users\sakok\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.48529412, 0.50246305, 0.47761194, 0.49253731, 0.47738693,
       0.53030303, 0.5255102 , 0.50510204, 0.5       , 0.51530612])

## Linear SVC

In [49]:
#let's run linear SVC next
text_clf_lsvc.fit(X_train, y_train)
#predictions
predsvc = text_clf_lsvc.predict(X_test)

In [69]:
df4 = pd.DataFrame(metrics.confusion_matrix(y_test,predsvc), 
      index=['James','John','Jude',
            'Luke','Mark','Matthew','Paul',
            'Peter','Unknown'], 
     columns=['James','John','Jude',
            'Luke','Mark','Matthew','Paul',
            'Peter','Unknown'])
df4

,James,John,Jude,Luke,Mark,Matthew,Paul,Peter,Unknown
James,3,2,0,7,3,2,23,1,0
John,1,332,0,60,12,27,34,1,1
Jude,0,0,0,2,0,0,3,2,0
Luke,1,56,0,473,55,67,57,1,5
Mark,1,21,0,75,52,61,11,0,0
Matthew,2,25,0,102,70,130,26,1,3
Paul,3,28,0,41,3,11,582,1,3
Peter,0,3,1,3,0,2,28,4,1
Unknown,0,14,0,15,0,6,44,1,22


In [71]:
print(metrics.classification_report(y_test,predsvc))

              precision    recall  f1-score   support

       James       0.27      0.07      0.12        41
        John       0.69      0.71      0.70       468
        Jude       0.00      0.00      0.00         7
        Luke       0.61      0.66      0.63       715
        Mark       0.27      0.24      0.25       221
     Matthew       0.42      0.36      0.39       359
        Paul       0.72      0.87      0.79       672
       Peter       0.33      0.10      0.15        42
     Unknown       0.63      0.22      0.32       102

   micro avg       0.61      0.61      0.61      2627
   macro avg       0.44      0.36      0.37      2627
weighted avg       0.59      0.61      0.59      2627



In [72]:
print(metrics.accuracy_score(y_test,predsvc))

0.6082984392843548


In [19]:
cross_val_score(text_clf_lsvc, X_test, y_test, cv=10)

c:\users\sakok\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.60784314, 0.57635468, 0.54228856, 0.55223881, 0.53768844,
       0.58080808, 0.56122449, 0.55102041, 0.56122449, 0.57653061])

## Logistic Regression

In [74]:
#run Naive Bayes first
text_clf_lr.fit(X_train, y_train)

#run predictions
predlr = text_clf_lr.predict(X_test)


c:\users\sakok\miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\sakok\miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [75]:
df5 = pd.DataFrame(metrics.confusion_matrix(y_test,predlr), 
      index=['James','John','Jude',
            'Luke','Mark','Matthew','Paul',
            'Peter','Unknown'], 
     columns=['James','John','Jude',
            'Luke','Mark','Matthew','Paul',
            'Peter','Unknown'])
df5

,James,John,Jude,Luke,Mark,Matthew,Paul,Peter,Unknown
James,0,2,0,4,0,1,34,0,0
John,0,295,0,95,5,16,56,0,1
Jude,0,1,0,2,0,0,4,0,0
Luke,0,39,0,540,7,30,99,0,0
Mark,0,23,0,116,30,33,19,0,0
Matthew,0,39,0,169,12,93,46,0,0
Paul,0,24,0,54,0,2,590,0,2
Peter,0,2,0,4,0,0,36,0,0
Unknown,0,14,0,25,0,1,58,0,4


In [76]:
print(metrics.classification_report(y_test,predlr))

c:\users\sakok\miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

       James       0.00      0.00      0.00        41
        John       0.67      0.63      0.65       468
        Jude       0.00      0.00      0.00         7
        Luke       0.54      0.76      0.63       715
        Mark       0.56      0.14      0.22       221
     Matthew       0.53      0.26      0.35       359
        Paul       0.63      0.88      0.73       672
       Peter       0.00      0.00      0.00        42
     Unknown       0.57      0.04      0.07       102

   micro avg       0.59      0.59      0.59      2627
   macro avg       0.39      0.30      0.29      2627
weighted avg       0.57      0.59      0.54      2627



In [77]:
print(metrics.accuracy_score(y_test,predlr))

0.5907879710696612


In [20]:
cross_val_score(text_clf_lr, X_test, y_test, cv=10)

c:\users\sakok\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
c:\users\sakok\miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\sakok\miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


array([0.54901961, 0.54187192, 0.52736318, 0.55223881, 0.55778894,
       0.57070707, 0.55102041, 0.54591837, 0.54591837, 0.55612245])

# Bag Of Words

In [21]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df



In [30]:
ntdoc = nlp(''.join(asv["Text"]))

In [31]:
# Set up the bags.
ntwords = bag_of_words(ntdoc)

In [ ]:
# Create our data frame with features.
word_counts = bow_features(df2, ntwords)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150


In [ ]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.25
                                                    )


print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

## Naive Bayes

In [ ]:
#run Naive Bayes first
nb = MultinomialNB()

nb.fit(X_train, y_train)

#run predictions
prednb = nb.predict(X_test)

In [ ]:
dfbownb = pd.DataFrame(metrics.confusion_matrix(y_test,prednb), 
      index=['James','John','Jude',
            'Luke','Mark','Matthew','Paul',
            'Peter','Unknown'], 
     columns=['James','John','Jude',
            'Luke','Mark','Matthew','Paul',
            'Peter','Unknown'])
dfbownb

In [ ]:
print(metrics.classification_report(y_test,prednb))

In [ ]:
print(metrics.accuracy_score(y_test,prednb))

In [ ]:
cross_val_score(nb, X_test, y_test, cv=10)

## Linear SVC

In [ ]:
#run Naive Bayes first
svc = LinearSVC()

svc.fit(X_train, y_train)

#run predictions
predsvc = svc.predict(X_test)

In [ ]:
dfbowsvc = pd.DataFrame(metrics.confusion_matrix(y_test,predsvc), 
      index=['James','John','Jude',
            'Luke','Mark','Matthew','Paul',
            'Peter','Unknown'], 
     columns=['James','John','Jude',
            'Luke','Mark','Matthew','Paul',
            'Peter','Unknown'])
dfbownsvc

In [ ]:
print(metrics.classification_report(y_test,predsvc))

In [ ]:
print(metrics.accuracy_score(y_test,predsvc))

In [ ]:
cross_val_score(svc, X_test, y_test, cv=10)

## Logistic Regression

In [ ]:
#run Naive Bayes first
lr = LogisticRegression()

lr.fit(X_train, y_train)

#run predictions
predlr = lr.predict(X_test)

In [ ]:
dfbowlr = pd.DataFrame(metrics.confusion_matrix(y_test,predlr), 
      index=['James','John','Jude',
            'Luke','Mark','Matthew','Paul',
            'Peter','Unknown'], 
     columns=['James','John','Jude',
            'Luke','Mark','Matthew','Paul',
            'Peter','Unknown'])
dfbowlr

In [ ]:
print(metrics.classification_report(y_test,predlr))

In [ ]:
print(metrics.accuracy_score(y_test,predlr))

In [ ]:
cross_val_score(lr, X_test, y_test, cv=10)